In [2]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

from circleguard import KeylessCircleguard, ReplayDir
from circleguard.judgment import JudgmentType
from slider import Library
from slider.beatmap import Circle, Slider

plt.style.use("ggplot")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

## Parse Notes

In [3]:
def compute_hitobject_angle(prev_obj, curr_obj, next_obj):
    
    EPSLION = 1e-06
    res = np.pi

    prev_vector = [ prev_obj.position.x - curr_obj.position.x, prev_obj.position.y - curr_obj.position.y ]
    next_vector = [ next_obj.position.x - curr_obj.position.x, next_obj.position.y - curr_obj.position.y ]

    prev_norm = (prev_vector[0] ** 2 + prev_vector[1] ** 2) ** 0.5
    next_norm = (next_vector[0] ** 2 + next_vector[1] ** 2) ** 0.5

    if prev_norm > EPSLION and next_norm > EPSLION:
        cos = (prev_vector[0] * next_vector[0] + prev_vector[1] * next_vector[1]) / (prev_norm * next_norm)
        res = np.arccos( np.clip(cos, -1, 1) )  

    return res


    
def get_hitobject_embedding(prev_obj, curr_obj, next_obj):
     

    x_position = curr_obj.position.x
    y_position = curr_obj.position.y

    in_x_offset = 0.0
    in_y_offset = 0.0
    in_distance = 0.0
    in_timedelta = 5000.0 # in ms

    out_x_offset = 0.0
    out_y_offset = 0.0
    out_distance = 0.0
    out_timedelta = 5000.0

    angle = np.pi

    is_slider = 0.0
    slider_duration = 0.0
    slider_length = 0.0
    slider_num_ticks = 0.0
    slider_num_beats = 0.0


    if prev_obj:
        in_x_offset = curr_obj.position.x - prev_obj.position.x
        in_y_offset = curr_obj.position.y - prev_obj.position.y
        in_distance = ( in_x_offset ** 2 + in_y_offset ** 2) ** 0.5
        in_timedelta = (curr_obj.time - prev_obj.time).microseconds / 1000

    if next_obj:
        out_x_offset = next_obj.position.x - curr_obj.position.x
        out_y_offset = next_obj.position.y - curr_obj.position.y
        out_distance = ( out_x_offset ** 2 + out_y_offset ** 2) ** 0.5
        out_timedelta = (next_obj.time - curr_obj.time).microseconds / 1000

    if prev_obj and next_obj:
        angle = compute_hitobject_angle(prev_obj, curr_obj, next_obj)

    if type(curr_obj) == Slider:
        is_slider = 1.0
        slider_duration = (curr_obj.end_time - curr_obj.time).microseconds / 1000
        slider_length = curr_obj.length
        slider_num_ticks = curr_obj.ticks
        slider_num_beats = curr_obj.num_beats

    return np.array([
        x_position, y_position,
        in_x_offset, in_y_offset, in_distance, in_timedelta,
        out_x_offset, out_y_offset, out_distance, out_timedelta,
        angle,
        is_slider, slider_duration, slider_length, slider_num_ticks, slider_num_beats
    ])



def filter_out_spinners(objects):
    return [o for o in objects if type(o) in (Slider, Circle)]



def get_embeddings(replay, beatmap_library, max_length = 2048):

    beatmap = beatmap_library.lookup_by_md5(replay.beatmap_hash)
    hitobjects = filter_out_spinners( beatmap.hit_objects() )

    res_len = min(len(hitobjects), max_length)
    res = np.zeros((res_len, 16))

    res[0, :] = get_hitobject_embedding(None, hitobjects[0], hitobjects[1])

    for emb_idx in range(1, res_len - 1):
        res[emb_idx, :] = get_hitobject_embedding( hitobjects[emb_idx - 1], hitobjects[emb_idx], hitobjects[emb_idx + 1] )

    res[res_len - 1 , :] = get_hitobject_embedding(hitobjects[res_len - 2], hitobjects[res_len - 1], None)

    return res

## Parse Judgments

In [4]:
def sort_judgments(judgments):
    return sorted(judgments, key = lambda j: j.hitobject.t)
    
def encode_judgment(judgment):
     t = judgment.type
     return np.array([
          1 if t == JudgmentType.Hit300 else 0, 
          1 if t == JudgmentType.Hit100 else 0,
          1 if t == JudgmentType.Hit50 else 0,
          1 if t == JudgmentType.Miss else 0
     ])

def get_judgments(replay, max_length = 2048):
    
     judgments = sort_judgments( cg.judgments(replay) )
     res_len = min( len(judgments), max_length )
     res = np.zeros((res_len, 4))

     # could probably be written w/o for loop?
     for judgment_idx in range(res_len):
          res[judgment_idx, :] = encode_judgment(judgments[judgment_idx])

     return res

## Parse Beatmap Info

In [5]:
def get_beatmap_context(replay, beatmap_library):

    beatmap = beatmap_library.lookup_by_md5(replay.beatmap_hash)
    mod_str = str(replay.mods)

    return np.array([
        
        beatmap.ar(),
        beatmap.od(),
        beatmap.cs(),
        beatmap.stack_leniency,

        1.0 if "EZ" in mod_str else 0.0,
        1.0 if "HD" in mod_str else 0.0,
        1.0 if "DT" in mod_str  or "NC" in mod_str else 0.0,
        1.0 if "HR" in mod_str else 0.0,

    ])

## Validate Replays

In [6]:
def validate_replay_length(replay, beatmap_objects, replay_judgments):

    res = 0

    beatmap_objects_no_spinners = filter_out_spinners(beatmap_objects)
    replay_objects = [ j.hitobject for j in replay_judgments ]

    if len(beatmap_objects_no_spinners) != len(replay_judgments):
        #print(f"Length mismatch between beatmap objects {(len(beatmap_objects_no_spinners))} and replay judgments {(len(replay_judgments))} of replay at {replay.path}.")
        res |= 1

    if len(beatmap_objects_no_spinners) != len(replay_objects):
        #print(f"Length mismatch between beatmap objects {(len(replay_objects))} and replay objects {(len(beatmap_objects))} of replay at {replay.path}.")
        res |= 2
    
    return res



def validate_replay_hitcounts(replay, beatmap_objects, replay_judgments):

    judgment_encodings = [encode_judgment(j) for j in replay_judgments]
    judgment_hitcounts = np.sum(judgment_encodings, axis = 0)
    replay_hitcounts = np.array([ replay.count_300, replay.count_100, replay.count_50, replay.count_miss])
    hitcount_err_arr = replay_hitcounts - judgment_hitcounts

    num_spinners = len(beatmap_objects) - len(filter_out_spinners(beatmap_objects))

    if sum(hitcount_err_arr) != num_spinners:
        #print(f"Hitcount mismatch ({judgment_hitcounts} vs. {replay_hitcounts}, num_spinners = {num_spinners}) of replay at {replay.path}.")
        return 4
    
    return 0



def validate_replay_objects(replay, beatmap_objects, replay_judgments):

    res = 0

    EPSILON = 1e-06
    beatmap_objects_no_spinners = filter_out_spinners(beatmap_objects)
    replay_objects = [ j.hitobject for j in replay_judgments ]

    for idx, _ in enumerate(beatmap_objects_no_spinners):

        replay_obj = replay_objects[idx]
        beatmap_obj = beatmap_objects_no_spinners[idx]

        if abs( replay_obj.time - beatmap_obj.time.total_seconds() * 1000 ) >= EPSILON : 
            #print(f"Offset mismatch ({1.0 * replay_obj.time} vs. {beatmap_obj.time.total_seconds() * 1000}) at index {idx} of replay at {replay.path}.")
            res |= 8

        if "HR" not in str(replay.mods) and "EZ" not in str(replay.mods): 

            if replay_obj.x - beatmap_obj.position.x >= EPSILON:
                #print(f"Position mismatch (x={replay_obj.x} vs. x={beatmap_obj.position.x}) at index {idx} of replay at {replay.path}.")
                res |= 16
            
            if replay_obj.y - beatmap_obj.position.y >= EPSILON:
                #print(f"Position mismatch (y={replay_obj.y} vs. y={beatmap_obj.position.y}) at index {idx} of replay at {replay.path}.")
                res |= 32
    
    return res
            



def validate_replay(replay, beatmap):

    res = 0

    beatmap_objects = beatmap.hit_objects()
    replay_judgments = sort_judgments( cg.judgments(replay) )

    res |= validate_replay_length(replay, beatmap_objects, replay_judgments)
    res |= validate_replay_hitcounts(replay, beatmap_objects, replay_judgments)
    res |= validate_replay_objects(replay, beatmap_objects, replay_judgments)
            
    return res


## Filter Replays

In [7]:
def filter_replay(replay, beatmap):

    res = 0
    
    mod_str = str(replay.mods)
    if "V2" in mod_str or "FL" in mod_str or "HT" in mod_str:
        res |= 1
    
    if replay.count_miss > 128:
        res |= 2
    
    if beatmap.max_combo < 128 or beatmap.max_combo > 8192:
        res |= 4
    
    if beatmap.bpm_min() < 32 or beatmap.bpm_max() > 512:
        res |= 8
    
    return res
    

## Write to File

In [8]:
def get_clean_mod_str(replay):
    mod_str = str(replay.mods)
    res =   "EZ" * ("EZ" in mod_str) + \
            "HD" * ("HD" in mod_str) + \
            "DT" * ("DT" in mod_str or "NC" in mod_str) + \
            "HR" * ("HR" in mod_str)
    if res == "":
        res = "NM"
    return res


def build_valid_index_df(replay_dir, beatmap_library):

    valid_idxs = []

    for replay_idx, replay in enumerate(replay_dir):

        beatmap = beatmap_library.lookup_by_md5(replay.beatmap_hash)

        if filter_replay(replay, beatmap) == 0 and validate_replay(replay, beatmap) == 0:
            valid_idxs.append([ replay_idx, get_clean_mod_str(replay), str(replay.path) ])

    valid_index_df = pd.DataFrame(valid_idxs, columns = ["replay_idx", "mods", "path"])

    return valid_index_df

In [9]:
tf_options = tf.io.TFRecordOptions(
    compression_type = "ZLIB",
)

def build_tf_example(replay, beatmap_library):

    embeddings = get_embeddings(replay, beatmap_library)
    judgments = get_judgments(replay)
    beatmap_context = get_beatmap_context(replay, beatmap_library)
    replay_length = len(embeddings)

    embeddings_feature = tf.train.Feature(
        float_list = tf.train.FloatList(
            value = embeddings.flatten()
        )
    )

    judgments_feature =  tf.train.Feature(
        float_list = tf.train.FloatList(
            value = judgments.flatten()
        )
    )

    beatmap_context_feature = tf.train.Feature(
        float_list = tf.train.FloatList(value = beatmap_context)
    )

    replay_length_feature = tf.train.Feature(
        int64_list = tf.train.Int64List(value = [replay_length])
    )

    feature = {
        "embeddings" : embeddings_feature,
        "judgments" : judgments_feature,
        "beatmap_context" : beatmap_context_feature,
        "replay_length" : replay_length_feature
    }

    return tf.train.Example(features = tf.train.Features(feature = feature))

## Driver Code

In [10]:
INDEX_PATH = "../data/indices/base_index.csv"
BEATMAP_PATH = "../data/beatmaps"
REPLAY_PATH = "../data/replays/osr"
DB_PATH = "../data/replays/osr/.circleguard.db"
SLIDER_PATH = "../data/beatmaps"

In [11]:
RANDOM_SEED = 12
BATCH_SIZE = 1024

In [12]:
# index and beatmaps

index_df = pd.read_csv(INDEX_PATH, low_memory = False)
beatmap_library = Library(BEATMAP_PATH)

print(f"Num. replays in index: {len(index_df)}")
print(f"Num. of beatmaps in library: {len(beatmap_library.ids)}")
index_df.head()

Num. replays in index: 381570
Num. of beatmaps in library: 55079


,Unnamed: 0,replayHash,beatmapHash,summary,date,playerName,modsReadable,mods,performance-IsFail,performance-Accuracy,performance-Score,performance-300s,performance-100s,performance-50s,performance-Misses,performance-Geki,performance-Katu,performance-MaxCombo,performance-IsFC,beatmap-Artist,beatmap-Title,beatmap-Version,beatmap-Id,beatmap-SetId,beatmap-BPMMin,beatmap-BPMMax,beatmap-HP,beatmap-OD,beatmap-AR,beatmap-CS,beatmap-MaxCombo,beatmap-HitObjects,beatmap-Circles,beatmap-Sliders,beatmap-Spinners,beatmapPlay-BPMMin,beatmapPlay-BPMMax,beatmapPlay-HP,beatmapPlay-OD,beatmapPlay-AR,beatmapPlay-CS,osrReplayUrl
0,0,857623324645f59599fc4e9c1c7e1130,db4cdde15984869de346686ceb6bc1a5,[7.44 ⭐] My Angel Shori | Imperial Circus Dead...,2022-11-11T08:03:56.656728,My Angel Shori,NaN,0,False,0.954368,28363840,1771,72,1,37,422,50,950,False,Imperial Circus Dead Decadence,Hyakki Yakou -Pandemonic Night Parade-,Youkai,3638479.0,1776682.0,134,235,5.0,9.2,9.7,4.5,2515,1881,1438,442,1,134.0,235.0,5.0,9.200000,9.700000,4.5,https://dl.issou.best/ordr/replays/cfc43370ca0...
1,1,b1675645282756f18e0c3f142f290d94,04129d1b26d6bf3f35cb00a04f4a8c88,[6.55 ⭐] AlexTheProtoTTV | katagiri - Buta Mus...,2022-11-11T04:37:52.235160,AlexTheProtoTTV,NaN,0,False,0.857687,2363246,991,174,10,50,274,103,194,False,katagiri,Buta Musou,AR 9.8,3777132.0,1647421.0,210,210,4.5,9.2,9.8,3.8,1660,1225,832,392,1,210.0,210.0,4.5,9.200000,9.800000,3.8,https://dl.issou.best/ordr/replays/e35f6f3e3b1...
2,2,6170131dcd9ad32d4c0287422b256423,8e3e4e77e6498a994bfa7505e735155d,[6.37 ⭐] Asio_ | Qrispy Joybox feat. mao - Col...,2022-11-11T08:01:02.823288,Asio_,HDDT,72,False,0.916667,777289,272,31,0,5,57,20,136,False,Qrispy Joybox feat. mao,Colorful Minutes,Beautiful Time,2584022.0,1242911.0,152,152,5.2,8.5,9.0,3.8,438,308,189,118,1,228.0,228.0,5.2,10.083333,10.333333,3.8,https://dl.issou.best/ordr/replays/f214715d45d...
3,3,d3fc501dadc1fabd60f847432cc3de84,27f9c5f8496bfdab8623e3be6cf73f25,[6.3 ⭐] sotarks fan123 | DragonForce - Valley ...,2022-11-11T07:46:29.042768,sotarks fan123,NaN,0,True,0.948815,45264320,1111,68,1,15,182,42,1485,False,DragonForce,Valley of the Damned,Apocalypse,675734.0,67565.0,71,200,6.0,8.5,9.2,4.0,3089,2148,1486,660,2,71.0,200.0,6.0,8.500000,9.200000,4.0,https://dl.issou.best/ordr/replays/cf3f6778f86...
4,4,8e62a939e4c97fe22cbd90e90d338a53,1e8f966c7a8f992cb2f3f5bab7d55925,[6.34 ⭐] XxSzymenxx | DragonForce - Through th...,2022-03-11T10:40:06.741672,XxSzymenxx,NaN,0,True,0.731128,486150,254,85,30,24,29,19,99,False,DragonForce,Through the Fire and Flames,Myth,1001682.0,382400.0,170,200,6.2,9.0,9.5,4.0,3220,2126,1534,587,5,170.0,200.0,6.2,9.000000,9.500000,4.0,https://dl.issou.best/ordr/replays/b90ed42a04b...


### Validation Loop

In [11]:
# START_BATCH = 0

# for batch_idx in range( START_BATCH, math.floor(len(index_df) / BATCH_SIZE) + 1 ):

#     cg = KeylessCircleguard( 
#         db_path = DB_PATH,
#         slider_dir = SLIDER_PATH,
#         cache = True
#     )
    
#     replay_dir = ReplayDir(REPLAY_PATH)
#     cg.load_info(replay_dir)

#     start_idx = batch_idx * BATCH_SIZE
#     end_idx = min( (batch_idx + 1) * BATCH_SIZE, len(replay_dir))

#     replay_dir.replays = replay_dir.replays[start_idx : end_idx]

#     try:
#         cg.load(replay_dir) # expensive
#         valid_replay_df = build_valid_index_df(replay_dir, beatmap_library)
#     except Exception as e:
#         print(f"Bad batch: {batch_idx}\n{e}")
#         continue

#     valid_replay_df["replay_idx"] = valid_replay_df["replay_idx"].apply(lambda idx: idx + start_idx)
#     valid_replay_df.to_csv(f"valid_index_df_{batch_idx}.csv", index = False)

### Dataset Split

In [12]:
# valid_index_df = pd.read_csv("data/valid_index.csv")

# X_train, X_test, _, _ = train_test_split(valid_index_df, valid_index_df["mods"], test_size = 0.1, random_state = RANDOM_SEED, stratify = valid_index_df["mods"])
# X_train, X_val, _, _  = train_test_split(X_train, X_train["mods"], test_size = 1/9, random_state = RANDOM_SEED, stratify = X_train["mods"])

# X_train.to_csv("train_index.csv", index = False)
# X_val.to_csv("val_index.csv", index = False)
# X_test.to_csv("test_index.csv", index = False)

### Write Loop

In [15]:
# START_BATCH = 0
# index_df = pd.read_csv("indices/test_index.csv")
# replay_indices = index_df["replay_idx"]

# for batch_idx in range( START_BATCH, math.floor(len(replay_indices) / BATCH_SIZE) + 1 ):

#     start_time = time.perf_counter()

#     cg = KeylessCircleguard( 
#         db_path = DB_PATH,
#         slider_dir = SLIDER_PATH,
#         cache = True
#     )
    
#     replay_dir = ReplayDir(REPLAY_PATH)
#     cg.load_info(replay_dir)

#     start_replay_idx = batch_idx * BATCH_SIZE
#     end_replay_idx = min( (batch_idx + 1) * BATCH_SIZE, len(replay_indices))
#     replay_idx_set = set( replay_indices[start_replay_idx : end_replay_idx] )
    
#     replay_dir.replays = [ replay_dir.replays[idx] for idx in replay_idx_set ]

#     try:
#         cg.load(replay_dir) # expensive
#     except Exception as e:
#         print(f"Bad batch: {batch_idx}\n{e}")
#         continue

#     writer = tf.io.TFRecordWriter(
#         path = f"test_{batch_idx}.tfrecord",
#         options = tf_options,
#     )

#     try:
#         for replay in replay_dir:
#             if "AT" not in str(replay.mods): # oops forgot about auto in the pre-processing
#                 tf_example = build_tf_example(replay, beatmap_library)
#                 writer.write(tf_example.SerializeToString())
#     except Exception as e:
#         print(f"Bad batch: {batch_idx}\n{e}")
#         writer.close()
#         continue

#     writer.close()

#     stop_time = time.perf_counter()
#     print(f"Completed batch {batch_idx} in {stop_time - start_time:0.4f} seconds.")

Completed batch 0 in 339.2798 seconds.
Completed batch 1 in 335.0701 seconds.
Completed batch 2 in 336.4882 seconds.
Completed batch 3 in 330.1502 seconds.
Completed batch 4 in 351.7655 seconds.
Completed batch 5 in 337.1833 seconds.
Completed batch 6 in 378.6550 seconds.
Completed batch 7 in 382.6547 seconds.
Completed batch 8 in 357.3149 seconds.
Completed batch 9 in 372.6471 seconds.
Completed batch 10 in 376.7967 seconds.
Completed batch 11 in 367.1809 seconds.
Completed batch 12 in 387.3293 seconds.
Completed batch 13 in 362.5147 seconds.
Completed batch 14 in 358.4064 seconds.
Completed batch 15 in 378.1597 seconds.
Completed batch 16 in 387.7082 seconds.
Completed batch 17 in 355.9399 seconds.
Completed batch 18 in 373.4390 seconds.
Completed batch 19 in 364.7807 seconds.
Completed batch 20 in 366.8308 seconds.
Completed batch 21 in 360.8402 seconds.
Completed batch 22 in 358.4149 seconds.
Completed batch 23 in 364.3526 seconds.
Completed batch 24 in 363.7298 seconds.
Completed 

### Example Read

In [16]:
# raw_data = tf.data.TFRecordDataset(
#     filenames = [
#         "data/records/test/test_1.tfrecord",
#         "data/records/test/test_14.tfrecord",
#         "data/records/test/test_25.tfrecord"
#     ],
#     compression_type = "ZLIB",
#     buffer_size = 10 * 1024, # 10MB
#     num_parallel_reads = 1,
# )

In [17]:
# feature_description = {
#     "embeddings": tf.io.VarLenFeature(dtype = tf.float32),
#     "judgments": tf.io.VarLenFeature(dtype = tf.float32),
#     "beatmap_context": tf.io.FixedLenFeature([8], dtype = tf.float32),
#     "replay_length": tf.io.FixedLenFeature([], dtype = tf.int64),
# }

In [21]:
# def parse_function(raw_data):
#   return tf.io.parse_single_example(raw_data, feature_description)

# parsed_data = raw_data.map(parse_function)
# parsed_data

In [20]:
# for replay in parsed_data:
#     print(replay["embeddings"].values)

### Write Visual Test Dataset

In [14]:
# NUM_REPLAYS = 1000
# index_df = pd.read_csv("indices/val_index.csv")
# replay_indices = index_df["replay_idx"]

# res = []

# cg = KeylessCircleguard( 
#     db_path = DB_PATH,
#     slider_dir = SLIDER_PATH,
#     cache = True
# )

# replay_dir = ReplayDir(REPLAY_PATH)
# cg.load_info(replay_dir)

# replay_dir.replays = replay_dir.replays[:NUM_REPLAYS]

# cg.load(replay_dir) # expensive

# writer = tf.io.TFRecordWriter(
#     path = f"visual.tfrecord",
#     options = tf_options,
# )

# for idx, replay in enumerate(replay_dir):
#     mod_str = str(replay.mods)
#     if "AT" not in mod_str and "V2" not in mod_str and "HR" not in mod_str and "DT" not in mod_str and "NC" not in mod_str and "EZ" not in mod_str:
        
#         beatmap = replay.beatmap(beatmap_library)
#         embs = get_embeddings(replay, beatmap_library)
        
#         if len(embs) >= 264:
            
#             stars = 0
#             try:
#                 stars = beatmap.stars()
#             except:
#                 continue
            
#             res.append([beatmap.display_name, str(replay.mods), stars])
                          
#             tf_example = build_tf_example(replay, beatmap_library)
#             writer.write(tf_example.SerializeToString())

# writer.close()

# pd.DataFrame(res, columns = ["beatmap_name", "mods", "stars_live"]).to_csv("visual_index.csv", index = False)